In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
tf.config.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip drive/MyDrive/data/GAN/laser-new.zip > /dev/null

In [4]:
image_train_gen = keras.preprocessing.image.ImageDataGenerator(width_shift_range=0.2,
                                                         height_shift_range=0.2,
                                                         rescale=1 / 255.,
                                                         horizontal_flip=True,
                                                         fill_mode="nearest",
                                                         validation_split=0.3
                                                         )

image_test_gen = keras.preprocessing.image.ImageDataGenerator(width_shift_range=0.2,
                                                         height_shift_range=0.2,
                                                         rescale=1 / 255.,
                                                         horizontal_flip=True,
                                                         fill_mode="nearest"
                                                         )

In [5]:
train_images = image_train_gen.flow_from_directory(
    directory="laser-new/",
    color_mode="rgb",
    class_mode="binary",
    subset='training',
    seed=42
    )

valid_images = image_train_gen.flow_from_directory(
    directory="laser-new/",
    color_mode="rgb",
    class_mode="binary",
    subset='validation',
    seed=42
    )

Found 9423 images belonging to 2 classes.
Found 4037 images belonging to 2 classes.


In [ ]:
test_images = image_test_gen.flow_from_directory(
    directory="scraper-new/",
    color_mode="rgb",
    class_mode="binary",
    seed=42
    )

Found 15534 images belonging to 2 classes.


In [ ]:
for _ in range(5):
    img, label = train_generator.next()
    #print(img.shape)   #  (1,256,256,3)
    #plt.imshow(img[0])
    #plt.show()

In [ ]:
train_images.image_shape

(256, 256, 3)

In [6]:
model = keras.models.Sequential([
    keras.layers.Conv2D(32, 5, padding="same", activation="relu", input_shape=(256,256,3)),
    keras.layers.MaxPooling2D(2),
    keras.layers.BatchNormalization(),    
    keras.layers.Dropout(0.2),
    keras.layers.Conv2D(64, 5, padding="same", activation="relu"),
    keras.layers.MaxPooling2D(2),
    keras.layers.BatchNormalization(),    
    keras.layers.Dropout(0.2),
    keras.layers.Conv2D(128, 5, padding="same", activation="relu"),
    keras.layers.MaxPooling2D(2),
    keras.layers.BatchNormalization(),    
    keras.layers.Dropout(0.2),
    keras.layers.Conv2D(256, 5, padding="same", activation="relu"),
    keras.layers.MaxPooling2D(2),
    keras.layers.BatchNormalization(),    
    keras.layers.Dropout(0.2),
    keras.layers.Flatten(),
    keras.layers.Dense(1, activation='sigmoid') ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      2432      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 32)     0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 128, 128, 32)     128       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 128, 128, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 64)      51264     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 64, 64, 64)       0

In [7]:
model.compile(loss="binary_crossentropy",
              optimizer=Adam(learning_rate=0.0002),
              metrics=["accuracy"])

In [8]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
checkpoint_cb = keras.callbacks.ModelCheckpoint("drive/MyDrive/models/GAN/laser-classifier-{epoch:02d}-{val_accuracy:.4f}.hdf5",  monitor='val_accuracy', save_best_only=True)

In [9]:
history = model.fit(train_images, epochs=100, validation_data=valid_images, callbacks=[early_stopping_cb, checkpoint_cb])

Epoch 1/100
295/295 [==============================] - 201s 638ms/step - loss: 0.0148 - accuracy: 0.9956 - val_loss: 2.5768 - val_accuracy: 0.7563
Epoch 2/100
295/295 [==============================] - 186s 631ms/step - loss: 0.0107 - accuracy: 0.9978 - val_loss: 3.3389e-09 - val_accuracy: 1.0000
Epoch 3/100
295/295 [==============================] - 187s 633ms/step - loss: 0.0074 - accuracy: 0.9988 - val_loss: 0.0017 - val_accuracy: 0.9993
Epoch 4/100
295/295 [==============================] - 187s 635ms/step - loss: 0.0031 - accuracy: 0.9996 - val_loss: 4.2389e-17 - val_accuracy: 1.0000
Epoch 5/100
295/295 [==============================] - 187s 635ms/step - loss: 0.0094 - accuracy: 0.9993 - val_loss: 1.3552e-04 - val_accuracy: 1.0000
Epoch 6/100
295/295 [==============================] - 187s 633ms/step - loss: 0.0042 - accuracy: 0.9994 - val_loss: 1.6961e-15 - val_accuracy: 1.0000
Epoch 7/100
295/295 [==============================] - 188s 638ms/step - loss: 0.0149 - accuracy: 0.99

KeyboardInterrupt: ignored